<a href="https://colab.research.google.com/github/M-H-Amini/GAN-Webinars/blob/main/SimpleGAN_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  In The Name Of ALLAH
#  Generative Adversarial Networks
#  PythonChallenge.ir
#  Mohammad Hossein Amini (mhamini@aut.ac.ir)
#  Lecture 0 - Simple GAN

In this lecture, we're going to implement a very simple GAN to generate some digits. We're focusing on the main concept of GANs, not the network architectures. So would use fully-connected networks (MLP) as the discriminator and generator structure. Let' see.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Flatten, InputLayer, Reshape

#  Dataset Preparation
We would use the famous **MNIST** dataset for handwritten digits. Let's assume we just want to generate a specific digit.

In [ ]:
(Xtrain, ytrain), (Xtest, ytest) = mnist.load_data()
Xtrain = Xtrain[ytrain==7]
ytrain = ytrain[ytrain==7]

#  Visualization
In order to visualize results, we would implement ```show``` function.

In [ ]:
def show(X, r=4, c=4):
  fig, ax = plt.subplots(r, c, True, True)
  for i in range(r):
    for j in range(c):
      ax[i][j].imshow(X[i * c + j])
  plt.show()

show(Xtrain)

We determine image shapes and the our noise dimension.

In [ ]:
img_shape = 28, 28
z_dim = 100

#  Discriminator
Let's build **discriminator** now. We use an MLP as the discriminator.

In [ ]:
def buildDisc(img_shape=(28, 28)):
  model = Sequential()
  model.add(InputLayer(img_shape))
  model.add(Flatten())
  model.add(Dense(64, activation='elu'))
  model.add(Dense(1, activation='sigmoid'))
  return model

disc = buildDisc()
disc.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

# Generator
Again, we use an MLP for the generator.

In [ ]:
def buildGen(img_shape=(28, 28), z_dim=100):
  model = Sequential()
  model.add(InputLayer((z_dim,)))
  model.add(Dense(64, 'elu'))
  model.add(Dense(img_shape[0]*img_shape[1], 'sigmoid'))
  model.add(Reshape(img_shape))
  return model

In [ ]:
gen = buildGen()

#  GAN
Time to build a model for **GAN**. We build it by cascading the generator and the discriminator.

In [ ]:
def buildGan(disc, gen):
  model = Sequential()
  model.add(gen)
  model.add(disc)
  return model

disc.trainable = False
gan = buildGan(disc, gen)
gan.compile(optimizer='adam', loss='binary_crossentropy')

#  Training

In [ ]:
for i in range(1000):
  indxs = np.random.permutation(len(Xtrain))[:16]
  Xreal = Xtrain[indxs]
  yreal = np.ones((16,))
  noise = np.random.normal(0, 1, size=(16, z_dim))
  Xfake = gen.predict(noise)
  yfake = np.zeros_like(yreal)

  ##  Disc. Training
  d_loss_real = disc.train_on_batch(Xreal, yreal)
  d_loss_fake = disc.train_on_batch(Xfake, yfake)
  d_loss, d_acc = np.add(d_loss_real, d_loss_fake) / 2

  ##  Gen. Training
  noise = np.random.normal(0, 1, size=(16, z_dim))
  g_loss = gan.train_on_batch(noise, yreal)

  if not ((i+1)%20):
    print(f'No {i + 1}\tD Acc: {d_acc}\tD Loss: {d_loss}\tG Loss: {g_loss}')
    Xfake = gen.predict(noise)
    show(Xfake)
